<a href="https://colab.research.google.com/github/abdallaho98/AutoSpeech/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install ktrain

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [0]:
import ktrain
from ktrain import text

In [7]:
import pandas as pd
filepath = 'legalts.txt'
df = pd.DataFrame(columns=('rating', 'review'))
with open(filepath) as fp:
   line = fp.readline()
   cnt = 0
   while line:
       x = line.strip().split("  ")
       df.loc[cnt] = [int(x[1]) , x[0]]
       line = fp.readline()
       cnt += 1
df['rating'] = df['rating'].apply(lambda x: 'neg' if x < 3 else 'pos')
df.columns = ['label', 'text']
df = pd.concat([df, df.label.astype('str').str.get_dummies()], axis=1, sort=False)
df = df[['text', 'neg', 'pos']]
df.head()

,text,neg,pos
0,يمكن أن ينصب الفحص الطبي على السوابق الوراثية ...,0,1
1,يمكن للمكتب، بقطع النظر عن أحكام المادة 4 أعلا...,0,1
2,(يمكن اكتساب الجنسية الجزائرية بالزواج من جزائ...,0,1
3,يمكن إنشاء فروع في دائرة اختصاص المحاكم على مس...,0,1
4,يمكن أحد الطرفين أن يفسخ العقد في أي وقت شريطة...,0,1


In [8]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df, 
                                                                   'text', # name of column containing review text
                                                                   label_columns=['neg', 'pos'],
                                                                   maxlen=500, 
                                                                   preprocess_mode='bert')

preprocessing train...
language: ar


preprocessing test...
language: ar


In [9]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500
done.


In [12]:
learner.fit_onecycle(2e-3, 3)



begin training using onecycle policy with max lr of 0.002...
Epoch 1/3
8/8 [==============================] - 251s 31s/step - loss: 1.1080 - accuracy: 0.5000 - val_loss: 0.7431 - val_accuracy: 0.5000
Epoch 2/3
8/8 [==============================] - 251s 31s/step - loss: 0.8080 - accuracy: 0.5000 - val_loss: 0.7008 - val_accuracy: 0.5000
Epoch 3/3
8/8 [==============================] - 250s 31s/step - loss: 0.8167 - accuracy: 0.5000 - val_loss: 0.7868 - val_accuracy: 0.5000
